In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt

from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP

np.random.seed(1337)
random.seed(1337)

# make up a dataset

from sklearn.datasets import make_moons, make_blobs
X, y = make_moons(n_samples=100, noise=0.1)

y = y*2 - 1 # make y be -1 or 1

# initialize a model 
model = MLP(2, [16, 16, 1]) # 2-layer neural network


# loss function
def loss(batch_size=None):
    
    # inline DataLoader :)
    if batch_size is None:
        Xb, yb = X, y
    else:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb, yb = X[ri], y[ri]
        
    inputs = [list(map(Value, xrow)) for xrow in Xb]
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # svm "max-margin" loss
    losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = data_loss + reg_loss
    
    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

# IBP Analysis on untrained network for a single input
#custom_x0 = 1
#custom_x1 = 0
#eps = 0.1
#input_with_bounds = [Value(custom_x0), Value(custom_x1)]
#input_with_bounds[0].lower = input_with_bounds[0].data - eps
#input_with_bounds[0].upper = input_with_bounds[0].data + eps
#input_with_bounds[1].lower = input_with_bounds[1].data - eps
#input_with_bounds[1].upper = input_with_bounds[1].data + eps
#score = model(input_with_bounds)
#score.ibp()
#print(f"Output bounds for custom input (untrained): lower={score.lower}, upper={score.upper}")

# optimization
for k in range(100):
    # forward
    total_loss, acc = loss()
    # backward
    model.zero_grad()
    total_loss.backward()
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad
    #if k % 1 == 0:
    #print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")

In [3]:
def computebounds(custom_x0, custom_x1, eps, model):
    input_with_bounds = [Value(custom_x0), Value(custom_x1)]
    input_with_bounds[0].lower = input_with_bounds[0].data - eps
    input_with_bounds[0].upper = input_with_bounds[0].data + eps
    input_with_bounds[1].lower = input_with_bounds[1].data - eps
    input_with_bounds[1].upper = input_with_bounds[1].data + eps

    score = model(input_with_bounds)
    score.ibp()
    return score


In [4]:
# Collect all ReLU nodes in the computation graph of the last score
def collect_relu_nodes(output_node):
    relu_nodes = []
    visited = set()
    def traverse(v):
          if v not in visited:
              visited.add(v)
              if getattr(v, '_op', None) == 'ReLU':
                  relu_nodes.append(v)
              for child in getattr(v, '_prev', []):
                  traverse(child)
    traverse(output_node)
    return relu_nodes

In [5]:
score = computebounds(0, 1, 0.1, model)
print(score.lower, score.upper)

-2.249150991889212 0.7945264909845504


In [ ]:
import copy
import random
 
score = computebounds(0, 1, 0.1, model)
        
relu_nodes = collect_relu_nodes(score)
        
        # 2. Pick a ReLU node at random
chosen_relu = random.choice(relu_nodes)
relu_input = list(chosen_relu._prev)[0]
branch_bounds = []
        
#copy relu_input deepcopy
# branch tree use copy for next branchesb
# gurobi

if relu_input.lower * relu_input.upper > 0:
    # Branch 1: ReLU input >= 0
    relu_input_new = relu_input
    relu_input_new.lower = max(0, relu_input.lower)
    relu_input_new.upper = max(0, relu_input.upper)
    relu_input_new.ibp() 
    branch_bounds.append((relu_input_new.lower, relu_input_new.upper))

    # Branch 2: ReLU input < 0
    relu_input.lower = 0
    relu_input.upper = 0
    relu_input.ibp() 
    branch_bounds.append((relu_input.lower, relu_input.upper))

else:
    relu_input.lower = max(0, relu_input.lower)
    relu_input.upper = max(0, relu_input.upper)
    relu_input.ibp() 
    branch_bounds.append((relu_input.lower, relu_input.upper))   


        # Compute global bounds
global_lower = min(b[0] for b in branch_bounds)
global_upper = max(b[1] for b in branch_bounds)
print(f"Global lower bound: {global_lower}, Global upper bound: {global_upper}")
        
        # branch_bounds contains the (lower, upper) for each subproblem"
      

Global lower bound: 0.1275869088739639, Global upper bound: 0.2155625067351223
